# Smart Optimization to Save Computing Time

In [ ]:
???

<table style="width:100%; height:90%">
      <tr>
    <th>Grid Search</th>
    <th>Smart Search</th>
  </tr>
  <tr>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%; height:100%"></td>
    <td><img src="src/08_Table_Smart Optimization.png" alt="Parametrize the Strategy" style="width:100%; height:100%"></td>
  </tr>
</table>

## Load the model

In [ ]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

## Load the data

In [ ]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

## Parametrize the Investment Strategy

### Import the strategy

In [ ]:
import strategies
from backtesting import Backtest

### Create Backtest class

In [ ]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [ ]:
bt = Backtest(df_explanatory, strategies.SimpleRegression,
              cash=10000, commission=.002, exclusive_orders=True)

### Optimize backtesting with multiple combinations

In [ ]:
import multiprocessing as mp
mp.set_start_method('fork')

In [ ]:
%%time

stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy=[0, 10],
    limit_sell=[-10, 0],
    maximize='Return [%]',
    method='skopt',
    max_tries=500,
    random_state=0,
    return_heatmap=True,
    return_optimization=True)

### Which are the best values for the parameters?

In [ ]:
heatmap

In [ ]:
dff = heatmap.reset_index()

In [ ]:
dff

In [ ]:
dff = dff.pivot(index='limit_buy', columns='limit_sell', values='Return [%]')

In [ ]:
dff

In [ ]:
import numpy as np

In [ ]:
dff.style.highlight_null(props='background-color: transparent; color: transparent')

In [ ]:
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient(vmin=np.nanmin(dff), vmax=np.nanmax(dff))\
    .highlight_null(props='background-color: transparent; color: transparent')

### How to interpret smart optimization?

#### Number of evaluations

https://kernc.github.io/backtesting.py/doc/examples/Parameter%20Heatmap%20&%20Optimization.html

In [ ]:
from skopt.plots import plot_evaluations

_ = plot_evaluations(optimize_result, bins=10)

#### Density

In [ ]:
from skopt.plots import plot_objective

_ = plot_objective(optimize_result, n_points=10)

https://kernc.github.io/backtesting.py/doc/examples/Multiple%20Time%20Frames.html